In [122]:
from dataclasses import dataclass, field
import numpy as np
from enum import Enum

class Direction(Enum):
    CLOCKWISE = 1
    COUNTERCLOCKWISE = 2

@dataclass
class FocusDistanceCalculator:
    min_deg: float = field()
    mid_deg: float = field()
    max_deg: float = field()
    range_deg: float = field(init=False)
    curr_deg: float = field(default=0, init=False)
    prev_deg: float = field(default=0, init=False)
    n: float = field(default=1, init=False)
    
    def __post_init__(self):
        self.prev_deg = self.max_deg
        self.range_deg = self.calculate_angular_range(self.min_deg, self.max_deg, self.mid_deg)
    
    def get_norm(self, new_angle: float) -> float:
        xt_new = new_angle * self.n
        xt_old = self.prev_deg
        
        if self.min_deg < xt_new < self.max_deg:
            pass
        else:
            direction = self.get_direction(xt_old, xt_new)
            if direction == Direction.CLOCKWISE:
                self.max_deg = xt_new
                self.min_deg = self.max_deg - self.range_deg
            elif direction == direction.COUNTERCLOCKWISE:
                self.min_deg = xt_new
                self.max_deg = self.min_deg + self.range_deg
        normed_value = (new_angle - self.min_deg) / self.range_deg
        self.prev_deg = new_angle
        return normed_value
        

    @staticmethod
    def calculate_angular_range(min_value: float, max_value: float, mid_value: float) -> float:
        direct_diff = (max_value - min_value) % 360
        wrapped_diff = (min_value - max_value) % 360

        # Determine which difference is correct based on the midpoint
        if (min_value < mid_value < max_value) or (max_value < min_value and (mid_value > min_value or mid_value < max_value)):
            return direct_diff
        else:
            return wrapped_diff
        
    @staticmethod
    def get_direction(xtold: float, xtnew: float) -> Direction:
        xtold = xtold % 360
        xtnew = xtnew % 360
        diff = (xtnew - xtold) % 360
        if diff < 180:
            return Direction.CLOCKWISE
        else:
            return Direction.COUNTERCLOCKWISE
    
    @staticmethod
    def _shortest_angle_distance(angle1: float, angle2: float) -> float:
        # Normalize the angles
        angle1 = angle1 % 360
        angle2 = angle2 % 360
        
        # Calculate the absolute difference
        diff = abs(angle1 - angle2)
        
        # Find the shortest distance
        shortest_distance = min(diff, 360 - diff)
        return shortest_distance
    
incoming_angles = np.array(
    np.linspace(10,  360, num=36).tolist() + 
    np.linspace(00,  350, num=36).tolist() +
    np.linspace(340, 0, num=35).tolist() +
    list(reversed(np.linspace(10, 350, num=35).tolist()))
)
print(incoming_angles)
fdc = FocusDistanceCalculator(min_deg=10, max_deg=150, mid_deg=80)
for angle in incoming_angles:
    print(f"θ={angle:} norm={fdc.get_norm(angle):.2f} min={fdc.min_deg:.2f} max={fdc.max_deg:.2f} range{fdc.range_deg}")
    
fdc = FocusDistanceCalculator(min_deg=10, max_deg=150, mid_deg=80)
incoming_angles = [15, 10, 5, 0, 5, 10]
print(incoming_angles)
for angle in incoming_angles:
    print(f"θ={angle:} norm={fdc.get_norm(angle):.2f} min={fdc.min_deg:.2f} max={fdc.max_deg:.2f} range{fdc.range_deg}")

# incoming_angles = [350, 360, 10, 20, 30]
# for angle in incoming_angles:
#     # xtnew = angle * n
#     fdc.get_norm(angle)
# 

# 1. test backward
# 2. test same 
# 3. test change under a threshold
# 4. 


[15, 10, 5, 0, 5, 10]
θ=15 norm=0.04 min=10.00 max=150.00 range140
θ=10 norm=0.00 min=10.00 max=150.00 range140
θ=5 norm=0.00 min=5.00 max=145.00 range140
θ=0 norm=0.00 min=0.00 max=140.00 range140
θ=5 norm=0.04 min=0.00 max=140.00 range140
θ=10 norm=0.07 min=0.00 max=140.00 range140


In [126]:
assert FocusDistanceCalculator.get_direction(10, 350) == Direction.COUNTERCLOCKWISE
assert FocusDistanceCalculator.get_direction(10, 20) == Direction.CLOCKWISE
assert FocusDistanceCalculator.get_direction(0, 179) == Direction.CLOCKWISE
assert FocusDistanceCalculator.get_direction(179, 0) == Direction.COUNTERCLOCKWISE
assert FocusDistanceCalculator.get_direction(350, 20) == Direction.CLOCKWISE
assert FocusDistanceCalculator.get_direction(350, 20) == Direction.CLOCKWISE


In [148]:
fdc = FocusDistanceCalculator(min_deg=10, max_deg=20, mid_deg=15)
assert np.isclose(fdc.get_norm(20), 1.0)
assert np.isclose(fdc.get_norm(15), (15 - 10) %360 / 10 )
assert np.isclose(fdc.get_norm(10), 0.0)
assert np.isclose(fdc.get_norm(5), 0.0)
assert np.isclose(fdc.get_norm(10), .5)
assert np.isclose(fdc.get_norm(14), (14 - 5) % 360 / 10)
assert np.isclose(fdc.get_norm(15), 1.0)